# Exploratory Data Analysis and Preparation
## Understanding and Preprocessing Data for Advanced Analytics
### Pablo X Zumba

The data set for this exercise includes information on house sales in King County, Washington (between May 2014 and May 2015). (Each row in the data set pertains to one house. There is a total of 21,613 houses in the data set). Use the data set to practice your DATA PREP skills. 
<br><br>
The data set is called "kc_house_data.csv".  


## Description of Variables

The description and type of each variable is provided in "KC house data - Data Dictionary.docx". Make sure to read this document to learn about the variables. Please check if variables are numeric, binary, or categorical.

# Setup

In [1]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(42)


# Get the data

In [2]:
# Import the data set:

king_county = pd.read_csv("kc_house_data.csv")
king_county.head()


,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,432000.0,5.0,2.75,2060.0,329903.0,1.5,0,3,5,7.0,2060,0,1989.0,0,98022.0,47.1776,-121.944,2240,220232.0
1,170000.0,2.0,1.00,810.0,8424.0,1.0,0,0,4,6.0,810,0,1959.0,0,98023.0,47.3286,-122.346,820,8424.0
2,235000.0,3.0,1.00,960.0,5030.0,1.0,0,0,3,7.0,960,0,1955.0,0,98118.0,47.5611,-122.280,1460,5400.0
3,350000.0,2.0,1.00,830.0,5100.0,1.0,0,0,4,7.0,830,0,1942.0,0,98126.0,47.5259,-122.379,1220,5100.0
4,397380.0,2.0,1.00,1030.0,5072.0,1.0,0,0,3,6.0,1030,0,1924.0,1958,98115.0,47.6962,-122.294,1220,6781.0


In [3]:
king_county.shape

(21613, 19)

# Split the data into train and test

In [4]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(king_county, test_size=0.3)

## Check the missing values

In [5]:
train.isna().sum()

price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         1
floors           1
waterfront       0
view             0
condition        0
grade            1
sqft_above       0
sqft_basement    0
yr_built         1
yr_renovated     0
zipcode          2
lat              0
long             0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [6]:
test.isna().sum()

price            0
bedrooms         1
bathrooms        0
sqft_living      1
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
zipcode          0
lat              0
long             0
sqft_living15    0
sqft_lot15       1
dtype: int64

# Data Prep

In [7]:
# Imports:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Separate the PRICE variable, which is the target variable (don't transform the target)

In [8]:
# Separate the target variable and input variables
train_targets = train[['price']]
test_targets = test[['price']]

train_inputs = train.drop(['price'], axis=1)
test_inputs = test.drop(['price'], axis=1)


##  Identify the numeric, binary, and categorical columns

In [9]:
train_inputs.dtypes

# Identify the numerical columns
numeric_columns = ['bathrooms','bedrooms', 'condition','floors','grade','lat','long','sqft_above','sqft_basement','sqft_living','sqft_living15','sqft_lot','sqft_lot15','view','yr_built','yr_renovated']

# Identify the categorical columns
categorical_columns = ['zipcode']

# Identify the binary columns so we can pass them through without transforming
binary_columns = ['waterfront']


In [10]:
binary_columns

['waterfront']

In [11]:
categorical_columns

['zipcode']

In [12]:
numeric_columns

['bathrooms',
 'bedrooms',
 'condition',
 'floors',
 'grade',
 'lat',
 'long',
 'sqft_above',
 'sqft_basement',
 'sqft_living',
 'sqft_living15',
 'sqft_lot',
 'sqft_lot15',
 'view',
 'yr_built',
 'yr_renovated']

# Pipeline (recommended)

If you don't want to use pipelines, feel free to use your own data prep steps.

In [13]:
# Numeric transformer:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [14]:
# Categorical transformer:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=99999)),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [15]:
# Binary transformer:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [16]:
# Column transformer:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns)],
        remainder='passthrough')

# Transform: fit_transform() for TRAIN

In [17]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x


<15129x88 sparse matrix of type '<class 'numpy.float64'>'
	with 257310 stored elements in Compressed Sparse Row format>

In [18]:
train_x.shape

(15129, 88)

# Tranform: transform() for TEST

In [19]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x


<6484x88 sparse matrix of type '<class 'numpy.float64'>'
	with 110274 stored elements in Compressed Sparse Row format>

In [20]:
test_x.shape

(6484, 88)

# Feature Names (optional)

You can get the feature names if you are running the new version of Scikit-Learn.

First, check your version of scikit-learn. If it is below 1.1.1, you might have to update it to the latest version as described below:

In [21]:
import sklearn

sklearn.__version__

'1.2.0'

In [22]:
# Since my version is 1.2.0, Now we can retrieve the column names:

preprocessor.get_feature_names_out()

array(['num__bathrooms', 'num__bedrooms', 'num__condition', 'num__floors',
       'num__grade', 'num__lat', 'num__long', 'num__sqft_above',
       'num__sqft_basement', 'num__sqft_living', 'num__sqft_living15',
       'num__sqft_lot', 'num__sqft_lot15', 'num__view', 'num__yr_built',
       'num__yr_renovated', 'cat__zipcode_98001.0',
       'cat__zipcode_98002.0', 'cat__zipcode_98003.0',
       'cat__zipcode_98004.0', 'cat__zipcode_98005.0',
       'cat__zipcode_98006.0', 'cat__zipcode_98007.0',
       'cat__zipcode_98008.0', 'cat__zipcode_98010.0',
       'cat__zipcode_98011.0', 'cat__zipcode_98014.0',
       'cat__zipcode_98019.0', 'cat__zipcode_98022.0',
       'cat__zipcode_98023.0', 'cat__zipcode_98024.0',
       'cat__zipcode_98027.0', 'cat__zipcode_98028.0',
       'cat__zipcode_98029.0', 'cat__zipcode_98030.0',
       'cat__zipcode_98031.0', 'cat__zipcode_98032.0',
       'cat__zipcode_98033.0', 'cat__zipcode_98034.0',
       'cat__zipcode_98038.0', 'cat__zipcode_98039.0',
    

In [24]:
# If you wanted, you can recreate the data set in the processed format:

king_county_train_processed = pd.DataFrame(train_x.toarray(), columns=preprocessor.get_feature_names_out())

king_county_train_processed

,num__bathrooms,num__bedrooms,num__condition,num__floors,num__grade,num__lat,num__long,num__sqft_above,num__sqft_basement,num__sqft_living,...,cat__zipcode_98155.0,cat__zipcode_98166.0,cat__zipcode_98168.0,cat__zipcode_98177.0,cat__zipcode_98178.0,cat__zipcode_98188.0,cat__zipcode_98198.0,cat__zipcode_98199.0,cat__zipcode_99999.0,binary__waterfront
0,0.175917,-0.399408,-0.627352,0.941617,-0.553271,-1.819530,-0.814467,-0.028098,-0.657178,-0.341652,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.445927,-1.460079,-0.627352,-0.911727,-1.406092,1.266441,-0.864098,-1.444027,-0.657178,-1.620032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.797190,0.661263,-0.627352,0.014945,-0.553271,0.908061,-0.743567,-0.094376,-0.657178,-0.401491,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.824654,-0.399408,-0.627352,0.941617,0.299550,0.418901,-0.552135,-0.106426,0.518347,0.153380,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.500286,0.661263,-0.627352,0.941617,0.299550,-0.180085,2.461144,0.484046,-0.657178,0.120741,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15124,-1.445927,-1.460079,-0.627352,-0.911727,-1.406092,1.129158,-0.502504,-1.058412,-0.657178,-1.271878,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15125,-1.445927,-0.399408,0.911430,-0.911727,-0.553271,-0.415634,0.284493,-0.757151,-0.657178,-0.999882,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15126,-1.445927,-0.399408,-0.627352,-0.911727,-0.553271,-0.962597,-0.771927,-0.720999,-0.657178,-0.967242,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15127,-0.472821,-0.399408,-2.166133,-0.911727,0.299550,0.462976,1.199112,-0.708949,0.111435,-0.586448,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
